In [3]:
import torch.nn as nn
import torch
import torch.nn.functional as F

THIS FILE IS  IMPLEMENTATION OF THE TRANSFORMER MODEL FROM SCRATCH WHITOUT DEPENDING ON PYTORCH PRE-BUILD CLASSES   AND THE TRAINING PROCCESS

In [1]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00


In [4]:


import tiktoken
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
# multihead attention from scratch
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, device=None):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.dk = d_model // num_heads
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.w_k = nn.Linear(d_model, d_model, bias=False).to(self.device)
        self.w_q = nn.Linear(d_model, d_model, bias=False).to(self.device)
        self.w_v = nn.Linear(d_model, d_model, bias=False).to(self.device)
        self.w_o = nn.Linear(d_model, d_model, bias=False).to(self.device)

    @staticmethod
    def attention(query, key, value, mask=None, padding_mask=None):
        dk = query.size(-1)
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(dk, dtype=torch.float32, device=query.device))

        if padding_mask is not None:
            padding_mask = padding_mask.unsqueeze(1).unsqueeze(1)
            attention_scores = attention_scores.masked_fill(padding_mask == 0, float('-inf'))

        if mask is not None:
            mask = mask.unsqueeze(1).unsqueeze(1)
            attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))

        attention_probs = F.softmax(attention_scores, dim=-1)
        attention_output = torch.matmul(attention_probs, value)

        return attention_output

    def forward(self, query, key, value, mask=None, padding_mask=None):
        batch_size = key.size(0)
        key_len, query_len = key.size(1), query.size(1)

        k = self.w_k(key)
        q = self.w_q(query)
        v = self.w_v(value)

        k = k.view(batch_size, key_len, self.num_heads, self.dk).transpose(1, 2)
        q = q.view(batch_size, query_len, self.num_heads, self.dk).transpose(1, 2)
        v = v.view(batch_size, key_len, self.num_heads, self.dk).transpose(1, 2)

        attention_output = self.attention(q, k, v, mask, padding_mask)

        attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, query_len, self.d_model)
        output = self.w_o(attention_output)

        return output


In [6]:
# the positional encoding add the positining of word in sentence
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000, device=None):
        super(PositionalEncoding, self).__init__()
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        pe = torch.zeros(max_len, d_model)  # No device specified here

        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe.to(self.device))  # Move pe to device
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [7]:
# the encoder block that use the multihead attention plus resutial connnection
class EncoderBlock(nn.Module):
    def __init__(self, n_embd, n_head, device=None):
        super().__init__()
        head_size = n_embd // n_head
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Initialize components
        self.sa = MultiHeadAttention(n_embd, n_head, device=self.device)
        self.ffwd = FeedForward(n_embd, device=self.device)  # Ensure FeedForward is CUDA-compatible
        self.ln1 = nn.LayerNorm(n_embd).to(self.device)
        self.ln2 = nn.LayerNorm(n_embd).to(self.device)

    def forward(self, x, mask):
        # Ensure input tensor x is on the correct device
        x = x.to(self.device)


        # Apply self-attention, add residual, and apply layer normalization
        x = x + self.ln1(self.sa(x, x, x, mask))

        # Apply feed-forward network, add residual, and apply layer normalization
        x = x + self.ln2(self.ffwd(x))

        return x

In [8]:
# the encoder repeat the proccess of EncoderBlock multiple times
class Encoder(nn.Module):
    def __init__(self, n_embd, n_head, n_layer, device=None):
        super().__init__()
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Create a list of EncoderBlock layers
        self.layers = nn.ModuleList([EncoderBlock(n_embd, n_head, device=self.device) for _ in range(n_layer)])

    def forward(self, x, mask=None):
        # Ensure input tensor x is on the correct device
        x = x.to(self.device)

        # Pass through each EncoderBlock
        for layer in self.layers:

            x = layer(x, mask)


        return x

In [9]:
# this is the decoder part of the transformer
class DecoderBlock(nn.Module):
    def __init__(self, n_embd, n_head, device=None):
        super().__init__()
        head_size = n_embd // n_head
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Initialize components
        self.mutli_head_attention = MultiHeadAttention(n_embd, 4, device=self.device)
        self.cross_attention = MultiHeadAttention(n_embd, 4, device=self.device)
        self.ffwd = FeedForward(n_embd, device=self.device)  # Ensure FeedForward is CUDA-compatible
    def forward(self, x, encoder_output,padding_mask):
      mask = torch.tril(torch.ones(1, x.shape[1])).to(self.device)

      x = x+self.mutli_head_attention(x,x,x,mask,padding_mask)

      x = x+self.cross_attention(x,encoder_output,encoder_output)
      x = x + self.ffwd(x)
      return x
        # Ensure input tensor x is on the correct device


In [10]:
# the decoder part of the transformer
class Decoder(nn.Module):
    def __init__(self, n_embd, n_head, n_layer, device=None):
        super().__init__()
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Create a list of EncoderBlock layers
        self.layers = nn.ModuleList([DecoderBlock(n_embd, n_head, device=self.device) for _ in range(n_layer)])
        self.n1  = nn.LayerNorm(n_embd).to(self.device)
    def forward(self, x, encoder_output,mask=None):
        # Ensure input tensor x is on the correct device
        x = x.to(self.device)
        for layer in self.layers:
            x = layer(x,encoder_output,mask)
        x = self.n1(x)
        return x

In [11]:
class FeedForward(nn.Module):
    def __init__(self, n_embd, device=None, dropout_prob=0.1):
        super().__init__()
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Define the feed-forward network with ReLU activation and dropout
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Dropout(dropout_prob),  # Apply dropout after ReLU
            nn.Linear(4 * n_embd, n_embd),
        ).to(self.device)  # Move the network to the specified device

    def forward(self, x):
        return self.net(x)

In [12]:
# feed forward to map emdadding to vocab size
class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)

In [13]:
# the full implementation fo the transformer model using only the elemnt we have build before
import torch.nn.functional as f
class Transformer(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, device=None):
        super().__init__()
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.sqrt = n_embd**0.5
        # Initialize components
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd).to(self.device)
        self.positional_encoding = PositionalEncoding(n_embd,0.2)
        self.encoder = Encoder(n_embd, n_head, n_layer, device=self.device)
        self.decoder = Decoder(n_embd, n_head, n_layer, device=self.device)
        self.lm_head = nn.Linear(n_embd, vocab_size).to(self.device)
        self.dropout = nn.Dropout(0.2)
        self.project  = ProjectionLayer(n_embd,vocab_size)

    def encoder_layer(self,x,mask):
        out =  self.encoder(x,mask)
        return out
    def decoder_layer(self,x,encoder_output,mask):
        out =  self.decoder(x,encoder_output,mask)
        return out
    def embadding(self,src):
         return  self.token_embedding_table(src)*self.sqrt

    def forward(self, src,target, mask=None):
      encoder_padding=(src != 50258).float()

      src = self.embadding(src)

      src  = self.positional_encoding(src)
      decoder_padding  = (target != 50258).float()
      target = self.embadding(target)

      target = self.positional_encoding(target)


      encoder_output = self.encoder_layer(src,encoder_padding)
      decoder_output = self.decoder_layer(target,encoder_output,decoder_padding)
      logits = self.project(decoder_output)

      return logits
    def inference(self, src, mask=None):

        tgt=torch.tensor([[50256]],device=self.device)
        src=torch.cat((torch.tensor([[50256]]), torch.tensor( [enc.encode(src)]),torch.tensor([[50257]])),dim=1).to(self.device)

        src= self.embadding(src)
        src=self.positional_encoding(src)
        out_encoder=self.encoder(src)


        while True:

            tgts= self.embadding(tgt)
            tgts=self.positional_encoding(tgts)
            logits=self.decoder(tgts,out_encoder)
            logits = self.project(logits)
            logits=logits[:,-1,:]


            logits = logits.view(-1)
            probs= f.softmax(logits,dim=0)

            prediction  = torch.argmax(probs)
            next_tgts=prediction.reshape(1,1)
            tgt=torch.cat((tgt,next_tgts),dim=1)


            print(enc.decode(tgt.squeeze(0).tolist()))
            if next_tgts.item()==50257:
                print(enc.decode(tgt.squeeze(0).tolist()))
                break
        return tgt



In [14]:
# this is huggingface librarie for datasets 
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [15]:
pip install tiktoken

In [16]:
from datasets import load_dataset
import tiktoken

In [17]:
# we use the same encoding as gpt 2 
enc = tiktoken.encoding_for_model("gpt2")

In [19]:
enc = tiktoken.Encoding(

    name="gpt2",
    pat_str=enc._pat_str,
    mergeable_ranks=enc._mergeable_ranks,
    special_tokens={
        "<sos>": 50256,
        "<eos>": 50257,
        "<pad>":50258
    }
)

In [21]:
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import torch

# Load the dataset from Hugging Face
raw_dataset = load_dataset("PaulineSanchez/Translation_words_and_sentences_english_french")['train'][:1000]

# Custom PyTorch Dataset to interface with the Hugging Face dataset
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):



        return {"english": enc.encode(self.dataset["English words/sentences"][idx],allowed_special="all"), "french":enc.encode(self.dataset["French words/sentences"][idx],allowed_special="all")}

# # Create the custom dataset
custom_dataset = CustomDataset(raw_dataset)
custom_dataset[0]

# # Define the DataLoader
batch_size = 32
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_data = CustomDataset(raw_dataset)

# # Function to generate random batches
def get_batch(batch_size=2):
    """
    Generate random batches with sequences dynamically padded to the same length.
    """
    # Randomly sample indices
    idx = torch.randint(1000, (batch_size,))

    # Encode the batches
    english_batch = [
        torch.tensor(train_data[i.item()]['english'])
        for i in idx
    ]
    french_batch = [
        torch.tensor(train_data[i.item()]['french'])
        for i in idx
    ]

    # Add special tokens (50258 for start, 50259 for end)
    trainer = [torch.cat((torch.tensor([50256]), tensor, torch.tensor([50257]))) for tensor in english_batch]
    decoder = [torch.cat((torch.tensor([50256]), tensor)) for tensor in french_batch]
    out = [torch.cat((tensor, torch.tensor([50257]))) for tensor in french_batch]

    # Determine the maximum sequence length for padding
    max_len = max(
        max(len(tensor) for tensor in trainer),
        max(len(tensor) for tensor in decoder),
        max(len(tensor) for tensor in out)
    )

    # Pad sequences to the maximum length
    input = pad_sequence(trainer, batch_first=True, padding_value=50258)
    output = pad_sequence(decoder, batch_first=True, padding_value=50258)
    out = pad_sequence(out, batch_first=True, padding_value=50258)

    return input.to(device), output.to(device), out.to(device)
get_batch()



(tensor([[50256,    51,  8056,   811,     0, 50257],
         [50256,  5189,  1781,    13, 50257, 50258]], device='cuda:0'),
 tensor([[50256,    38, 35942,   498,   447,   107,     0],
         [50256,    33,  2013,   264, 42324,    81,    13]], device='cuda:0'),
 tensor([[   38, 35942,   498,   447,   107,     0, 50257],
         [   33,  2013,   264, 42324,    81,    13, 50257]], device='cuda:0'))

In [22]:
# intialise the transformer model
english_french_transformer =  Transformer(enc.n_vocab,512,8,8)
encoder,decoder,target  =  get_batch()
mask = torch.tril(torch.ones((encoder.shape[1], encoder.shape[1])))
english_french_transformer = english_french_transformer.to(device)

In [23]:
# calculate the number of parameters in the model
pytorch_total_params = sum(p.numel() for p in english_french_transformer.parameters())
print(pytorch_total_params)

136076966


In [24]:
# enpty the gpu
torch.cuda.empty_cache()


In [ ]:
# i would recommand that you use weight and baies to visualise you data progress

In [25]:
from tqdm import tqdm

# Define optimizer
optimizer = torch.optim.AdamW(english_french_transformer.parameters(), lr=1e-4)
# vocab_size = 50
# Wrap the loop with tqdm for progress bar
progress_bar = tqdm(range(1000), desc="Training Progress", unit="batch")
english_french_transformer.train()
for i in progress_bar:
    x , y , z = get_batch(30)  # Get a batch of training data
    logits= english_french_transformer(x, y)  # Forward pass

    B , T,C = logits.shape
    logits = logits.view(B*T, C)
    targets = z.view(B*T)
    loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad(set_to_none=True)  # Reset gradients
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    # Dynamically display the loss at each iteration
    progress_bar.set_postfix({"Loss": loss.item()})


Training Progress:  16%|█▋        | 165/1000 [00:19<01:38,  8.44batch/s, Loss=1.93]


KeyboardInterrupt: 

In [ ]:
print(targets,logits)

tensor([40932,   279,  1072,   627,     6,   346,  1556,  2169,   862, 12797,
         6941,    72,   288,     6,  3258, 25792,   353, 12797,   257, 23577,
          454,    67,     6,    71,  9019,    13, 50257], device='cuda:0') tensor([[-5.5289, -4.9186, -4.6765,  ..., -5.9165,  8.0270, -4.5526],
        [-5.5071, -4.9121, -4.8158,  ..., -5.8610,  7.9478, -4.5072],
        [-5.5314, -4.9203, -4.7465,  ..., -5.8397,  7.9506, -4.5072],
        ...,
        [-5.5570, -4.9310, -4.7348,  ..., -5.8437,  7.9310, -4.5126],
        [-5.6017, -4.9109, -4.7667,  ..., -5.8244,  7.9770, -4.5137],
        [-5.6004, -4.8817, -4.7502,  ..., -5.8522,  7.9142, -4.5155]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [26]:
# use this  for inference after training
english_french_transformer.eval()
print(enc.decode(x[0].tolist()))
print(enc.decode(y[0].tolist()))
print(enc.decode(z[0].tolist()))
sentence =enc.decode(x[0].tolist())
english_french_transformer.inference(sentence)

<sos>Stop that!<eos><pad>
<sos>Arrêtez !<pad><pad><pad><pad><pad><pad><pad><pad><pad>
Arrêtez !<eos><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<sos>C
<sos>C'
<sos>C'est
<sos>C'est-
<sos>C'est-mo
<sos>C'est-moi
<sos>C'est-moi !
<sos>C'est-moi !<eos>
<sos>C'est-moi !<eos>


tensor([[50256,    34,     6,   395,    12,  5908,    72,  5145, 50257]],
       device='cuda:0')